In [170]:
import pandas as pd
import numpy as np
from ast import literal_eval
import requests
from configs import api_key

### Extracting data from first Movielens dataset

In [86]:
movie_ml_25=pd.read_csv('../Resources/ml-25m/movies.csv')
movie_ml_25.head(2)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy


In [87]:
link_ml_25=pd.read_csv('../Resources/ml-25m/links.csv')
link_ml_25.head(2)

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0


In [88]:
link_ml_25.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62423 entries, 0 to 62422
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  62423 non-null  int64  
 1   imdbId   62423 non-null  int64  
 2   tmdbId   62316 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 1.4 MB


### Merging dataframes

In [89]:
ml_25_df = pd.merge(link_ml_25, movie_ml_25, on="movieId")
ml_25_df.head(2)

,movieId,imdbId,tmdbId,title,genres
0,1,114709,862.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,113497,8844.0,Jumanji (1995),Adventure|Children|Fantasy


In [90]:
ml_25_df.isna().sum()

movieId      0
imdbId       0
tmdbId     107
title        0
genres       0
dtype: int64

In [91]:
#extract year and title from title column
ml_25_df = ml_25_df.assign(year = lambda x: x['title'].str.extract('(\d+)'))
ml_25_df = ml_25_df.assign(title = lambda x: x['title'].str.split('(').str[0])
ml_25_df.head(2)


,movieId,imdbId,tmdbId,title,genres,year
0,1,114709,862.0,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,113497,8844.0,Jumanji,Adventure|Children|Fantasy,1995


In [92]:
ml_25_df = ml_25_df.assign(genres = lambda x: x['genres'].str.split('|'))

In [93]:
#checking null values after extracting
ml_25_df.drop(['tmdbId'],axis=1,inplace=True)
ml_25_df.isna().sum()

movieId      0
imdbId       0
title        0
genres       0
year       387
dtype: int64

In [94]:
len(ml_25_df)

62423

### Extracting data from metadata Movielens json 

In [101]:
json_metadata=pd.read_json('../Resources/movie_dataset_public_final/raw/metadata_updated.json',orient='str', lines=True)
json_metadata.head(2)

,title,directedBy,starring,avgRating,imdbId,item_id
0,Toy Story (1995),John Lasseter,"Tim Allen, Tom Hanks, Don Rickles, Jim Varney,...",3.89146,114709,1
1,Jumanji (1995),Joe Johnston,"Jonathan Hyde, Bradley Pierce, Robin Williams,...",3.26605,113497,2


In [102]:
#extract year and title from title column
json_metadata = json_metadata.assign(year = lambda x: x['title'].str.extract('(\d+)'))
json_metadata = json_metadata.assign(title = lambda x: x['title'].str.split('(').str[0])
json_metadata.head(2)

,title,directedBy,starring,avgRating,imdbId,item_id,year
0,Toy Story,John Lasseter,"Tim Allen, Tom Hanks, Don Rickles, Jim Varney,...",3.89146,114709,1,1995
1,Jumanji,Joe Johnston,"Jonathan Hyde, Bradley Pierce, Robin Williams,...",3.26605,113497,2,1995


In [103]:
#checking null values after extracting
json_metadata.isna().sum()

title           0
directedBy      0
starring        0
avgRating       0
imdbId          0
item_id         0
year          618
dtype: int64

### Merging Movielense dataframes 

In [108]:
df=json_metadata[['imdbId','directedBy','starring','avgRating']]
mvielense_df = pd.merge(ml_25_df, df, on=["imdbId"],how='inner')
mvielense_df.head(2)

,movieId,imdbId,title,genres,year,directedBy,starring,avgRating
0,1,114709,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,John Lasseter,"Tim Allen, Tom Hanks, Don Rickles, Jim Varney,...",3.89146
1,2,113497,Jumanji,"[Adventure, Children, Fantasy]",1995,Joe Johnston,"Jonathan Hyde, Bradley Pierce, Robin Williams,...",3.26605


In [162]:
len(mvielense_df)

62417

In [110]:
#checking null values after extracting
mvielense_df.isna().sum()

movieId         0
imdbId          0
title           0
genres          0
year          386
directedBy      0
starring        0
avgRating       0
dtype: int64

### Using OMDB API for imputing null value of the year

In [232]:
#miss_year_movie=mvielense_df[mvielense_df['year'].isna()]['title'].to_list()
#url = "http://www.omdbapi.com/?apikey=" + api_key + "&t="
#Title = []
#Year = []
#imdbId=[]

#for i in miss_year_movie:
    #try:
       # response = requests.get(url + i).json()
    
       # Title.append(response['Title'])
       # Year.append(response['Year'])
       # imdbId.append(response['imdbID'])
   # except:
      #  print("title not found")
    

In [228]:
#OMDB movies data
impute_df=pd.DataFrame({'title':Title,'year':Year})
impute_df.head(2)

,title,year
0,Terrible Joe Moran,1984
1,The Court-Martial of Jackie Robinson,1990


In [229]:
#The rows with null value in year column
year_miss_df=mvielense_df[mvielense_df['year'].isna()].drop(['year'],axis=1)
year_miss_df.head(2)

,movieId,imdbId,title,genres,directedBy,starring,avgRating
15036,79607,66079,"Millions Game, The","[Action, Drama, Sci-Fi, Thriller]",Tom Toelle,"Jörg Pleva, Dieter Hallervorden, Josef Fröhlic...",3.625
25387,123619,88248,Terrible Joe Moran,[(no genres listed)],,,2.500


In [278]:
#imputing rows by merging the api data for year column
imputing_df = pd.merge(year_miss_df, impute_df, on=["title"],how='left')
imputing_df = imputing_df.reindex(columns=['movieId', 'imdbId', 'title', 'genres', 'year', 'directedBy',
       'starring', 'avgRating'])
imputing_df.head(2)

,movieId,imdbId,title,genres,year,directedBy,starring,avgRating
0,79607,66079,"Millions Game, The","[Action, Drama, Sci-Fi, Thriller]",NaN,Tom Toelle,"Jörg Pleva, Dieter Hallervorden, Josef Fröhlic...",3.625
1,123619,88248,Terrible Joe Moran,[(no genres listed)],1984,,,2.500


In [279]:
imputing_df.isna().sum()

movieId         0
imdbId          0
title           0
genres          0
year          105
directedBy      0
starring        0
avgRating       0
dtype: int64

In [280]:
mvielense_df=mvielense_df.dropna()
imputing_df=imputing_df.dropna()
clean_movie_df = pd.concat([mvielense_df, imputing_df])
clean_movie_df.head(2)

,movieId,imdbId,title,genres,year,directedBy,starring,avgRating
0,1,114709,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,John Lasseter,"Tim Allen, Tom Hanks, Don Rickles, Jim Varney,...",3.89146
1,2,113497,Jumanji,"[Adventure, Children, Fantasy]",1995,Joe Johnston,"Jonathan Hyde, Bradley Pierce, Robin Williams,...",3.26605


In [281]:
clean_movie_df.isna().sum()

movieId       0
imdbId        0
title         0
genres        0
year          0
directedBy    0
starring      0
avgRating     0
dtype: int64

In [282]:
len(clean_movie_df)

62312

In [301]:
clean_movie_df.to_csv('output_files/clean_movie.csv',index=False)